In [88]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-campaign/marketing_campaign.csv


In [89]:
!pip install plotly

# **Introduction**

**Customer personality analysis is a detailed analysis of a company's ideal customers. It helps a business to better understands its customers and makes it easier for them to modify products according to the specific needs, behaviours and concerns of different types of customers.****

**Customer personality analysis helps a bsiness to modify its products based on its target customers from different types of customer segments. Instead of spending money to market a new product to every customer in the company's database, a company can analize which customer segment is most likely to buy the product and then market the product only on that particular segment.****



In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import datetime
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.subplots as sp

import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/marketing-campaign/marketing_campaign.csv


In [91]:
df = pd.read_csv('/kaggle/input/marketing-campaign/marketing_campaign.csv', sep='\t')
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [93]:
df.shape

(2240, 29)

In [94]:
df.describe

<bound method NDFrame.describe of          ID  Year_Birth   Education Marital_Status   Income  Kidhome  \
0      5524        1957  Graduation         Single  58138.0        0   
1      2174        1954  Graduation         Single  46344.0        1   
2      4141        1965  Graduation       Together  71613.0        0   
3      6182        1984  Graduation       Together  26646.0        1   
4      5324        1981         PhD        Married  58293.0        1   
...     ...         ...         ...            ...      ...      ...   
2235  10870        1967  Graduation        Married  61223.0        0   
2236   4001        1946         PhD       Together  64014.0        2   
2237   7270        1981  Graduation       Divorced  56981.0        0   
2238   8235        1956      Master       Together  69245.0        0   
2239   9405        1954         PhD        Married  52869.0        1   

      Teenhome Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  \
0            0  04-09-2012  

In [95]:
df["Age"] = datetime.datetime.now().year - df.Year_Birth
bins = list(range(0, 110, 10)) + [np.inf]
print(bins)
# Definning the corresponding labels
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', 'Over 100']

# Customers Age Group
df["Age_Group"] = pd.cut(df.Age, bins, labels=labels)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, inf]


In [96]:
df[df.isnull().any(axis=1)]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age,Age_Group
10,1994,1983,Graduation,Married,NaN,1,0,15-11-2013,11,5,...,0,0,0,0,0,3,11,0,41,41-50
27,5255,1986,Graduation,Single,NaN,1,0,20-02-2013,19,5,...,0,0,0,0,0,3,11,0,38,31-40
43,7281,1959,PhD,Single,NaN,0,0,05-11-2013,80,81,...,0,0,0,0,0,3,11,0,65,61-70
48,7244,1951,Graduation,Single,NaN,2,1,01-01-2014,96,48,...,0,0,0,0,0,3,11,0,73,71-80
58,8557,1982,Graduation,Single,NaN,1,0,17-06-2013,57,11,...,0,0,0,0,0,3,11,0,42,41-50
71,10629,1973,2n Cycle,Married,NaN,1,0,14-09-2012,25,25,...,0,0,0,0,0,3,11,0,51,51-60
90,8996,1957,PhD,Married,NaN,2,1,19-11-2012,4,230,...,0,0,0,0,0,3,11,0,67,61-70
91,9235,1957,Graduation,Single,NaN,1,1,27-05-2014,45,7,...,0,0,0,0,0,3,11,0,67,61-70
92,5798,1973,Master,Together,NaN,0,0,23-11-2013,87,445,...,0,0,0,0,0,3,11,0,51,51-60
128,8268,1961,PhD,Married,NaN,0,1,11-07-2013,23,352,...,0,0,0,0,0,3,11,0,63,61-70


From above result we can observe that the dataset has no missing values in the columns except for the "Income" column. Also noticable is that the "Z_CostContact" and "Z_Revenue" columns appear to have constant values.



**Customers Demographics**
Customer demographics comprise the objective and subjective qualities that define a particular group of buyers. Analyzing customer demographics is essential in customer personality analysis as it provides valuable insights into the characteristics, preferences, and behaviors of different customer segments.

In [97]:
cusomers_age_group = df.Age_Group.value_counts().reset_index()

cusomers_age_group.columns = ["Age_Group", "Count"]
                                            
fig = px.bar(cusomers_age_group, x="Age_Group", y="Count")
fig.show()

In [98]:
customers_age_ms_group = df.groupby(["Age_Group", "Marital_Status"]).size()\
                                                                .reset_index(name="Count")

target_age_groups = ["31-40", "41-50", "51-60", "61-70", "71-80"]

fig = px.bar(customers_age_ms_group.loc[customers_age_ms_group.Age_Group.isin(target_age_groups), :], 
             x="Age_Group", y="Count", color="Marital_Status", barmode="group")

fig.show()

In [99]:
# Calculate the average values for each age group
avg_childrenhome = df.groupby(["Age_Group"]).agg({"Kidhome":np.mean, "Teenhome":np.mean})\
                            .reset_index()


# Create the bar traces
def bar_avg_children(data, col, name):
    data_copy = data.copy().loc[data.Age_Group.isin(target_age_groups)]
    
    children_trace = go.Bar(
        x=data_copy.Age_Group,
        y=data_copy[f"{col}"],
        name=name
    )
    
    return children_trace
    
kidhome_trace = bar_avg_children(avg_childrenhome, "Kidhome", "Average Num Of Kids per Household") 
teenhome_trace = bar_avg_children(avg_childrenhome, "Teenhome", "Average Num Of Teens per Household") 

# Create the layout
layout = go.Layout(
    xaxis=dict(title='Age Group'),
    yaxis=dict(title='Average Number of Children')
)

# Create the figure
fig = go.Figure(data=[kidhome_trace, teenhome_trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
fig = px.histogram(df.Age, marginal="box")
fig.show()

print(f"Skewness of Age Distribution is {df.Age.skew()}")

Central tendecy of customers age:

Median Age: 53 y.o
Mean Age: 54 y.o (considering we have a few clear outliers when deriving age from the year of birth)

In [ ]:
fig = px.histogram(df.Income, marginal="box", nbins=100)
fig.update_layout(xaxis_range=[0, 120_000]) # there are a few outliers above 120K income
fig.show()

Based on the histogram and skewness measures, it appears that the data is relatively symmetric and not strongly skewed in any particular direction. But outliers should be taken in consideration when defining central tendency of income distribution.

There are only a few outliers that's why mean doesn't differ substantially from the median

Central tendecy of customers income:

Median Income: 51381.50 USD
Mean Income: 52247.25 USD

Income Distribution by Education Category
This section would focus on analyzing and understanding the distribution of income across various education levels within the dataset.

In [ ]:
fig = px.histogram(df, x="Income", color="Education")
fig.update_layout(xaxis_range=[0, 120_000])
fig.show()

In [ ]:
fig = px.histogram(df, x="Income",
                   facet_col="Education")
fig.update_layout(xaxis_range=[0, 120_000], yaxis_range=[0,120])
fig.show()

The analysis indicates that there is a notable similarity in income distribution among individuals holding various graduate degrees, including Graduated, PhD, Master, and 2nd Cycle degrees. However, it is important to acknowledge that the representation of these educational levels differs. This discrepancy can be attributed to the inherent complexity associated with attaining higher degrees and educational accomplishments.

In [ ]:
# Convert "Dt_Customer" column to datetime format
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)

# Calculate today's date
today = pd.to_datetime('today')

# Calculate the duration in years since enrollment
customers_data['Enrollment_Duration_Years'] = ((today - df['Dt_Customer']).dt.days // 365)

print("Min Enrollment Duration is {} (Years)".format(df.Enrollment_Duration_Years.min()))
df.loc[:, "Age":].head()

*The dataset encompasses Customers with a minimum purchase history of 8 years or longer, demonstrating their enduring engagement with the company's products.*

*It is a crucial section in our comprehensive customer personality analysis. It involves identifying the top-selling position, gaining valuable insights into its success, and optimizing other product offerings. We carefully select appropriate statistical measures to accurately interpret the data and uncover key factors influencing customers' purchasing decisions.*

In [ ]:
product_columns = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds']

product_names = ["Wines", "Fruit", "Meat Products", "Fist Products", "Sweet Products", "Gold Products"]

In [ ]:
traces = [go.Histogram(x=df[product_col], name=product_col) for i, product_col in enumerate(product_columns)]

fig = sp.make_subplots(rows=2, cols=3, 
                       subplot_titles=product_names)

trace_index = 0
for r in range(1, 3):
    for c in range(1, 4):
            fig.add_trace(traces[trace_index], row=r, col=c)
            trace_index=1
            
# Set the margins for the subplots
fig.update_layout(
    margin=dict(l=20, r=20, t=100, b=40),  # Set left, right, top, and bottom margins
    title="Product Purchases Distributions",
)
                       
fig.show()                       

*Distribution plots indicate skewed to the right distributions of features. Thus, it is crucial to avoid assuming that a product (e.g., Wines) is the top-selling item based solely on individual customers who purchase more of it.*

In [ ]:
melt_columns = {"variable": "Product Type", "value": "Meadian Purchases"}



total_num_of_purchased_products_per_category = pd.melt(df.loc[:, product_columns])\
                                                .rename(melt_columns, axis=1)\
                                                .groupby("Product Type").median()\
                                                .sort_values(by="Meadian Purchases", ascending=False) 
                                                

fig = px.bar(total_num_of_purchased_products_per_category)
fig.show()

*While it is evident that Wines Product holds the top-selling position, it is noteworthy that the median selling volume is considerably lower than the average (173 median vs 303 mean). This serves as a crucial reminder to avoid overestimating the profitability of certain products.*

In [ ]:
top_selling_products = ["MntWines", "MntMeatProducts", "MntGoldProds", "MntFishProducts"]

*Product vs Product
Within this section, we will explore the correlation between different products to determine if the purchase of one item influences the purchase of others.*

In [ ]:
def correlation_triangle_heatmap(data):
    corrmat = data.corr()
    mask = np.triu(np.ones_like(data.corr(), dtype=bool))
    heatmap = sns.heatmap(corrmat, vmax=.8, annot=True ,mask=mask)
    return heatmap

correlation_triangle_heatmap(df.loc[:, product_columns])

*Purchases of all products categories are moderately correlated with each other. Some pairs such as Fruits and Fish or Fish and Sweet or Fruits and Sweet, etc are more correlated than others. This observation can be important in following analysis.*

*We will explore the correlation between different products and Income to determine if income influences the purchase of certain goods*

In [ ]:
def correlation_with_products_heatmap(source_data, columns):
    columns_to_load = product_columns + columns

    data = source_data.copy().loc[:, columns_to_load]
    
    corrmat = data.corr().sort_values(by=columns, ascending=False)
    
    corrmat.drop(columns, axis=1, inplace=True)
    corrmat.drop(data.columns[:-len(columns)], inplace=True)
    
    heatmap = sns.heatmap(corrmat, vmax=.8, annot=True)
    return heatmap

correlation_with_products_heatmap(df, columns=["Income", "Age"])

In [ ]:
def get_product_props_per_variable(col_name):
    products_education_data = df.copy().loc[:, product_columns+[col_name]]
    products_education_data_melted = products_education_data.melt(id_vars=col_name, var_name="Product", value_name="Sum")\
                                                                  .groupby([col_name, "Product"]).sum()

    pivot_table = products_education_data_melted.unstack(level=0)
    pivot_table["Total"] = pivot_table.sum(axis=1)
    pivot_table = pivot_table.div(pivot_table['Total'], axis=0)
    pivot_table.drop("Total", axis=1, inplace=True)
    
    return pivot_table.loc[:, "Sum"]

In [ ]:
df_education = get_product_props_per_variable("Education")
    
fig = px.bar(df_education, barmode='stack')
fig.show()

In [ ]:
df_education = get_product_props_per_variable("Marital_Status")
    
fig = px.bar(df_education, barmode='stack')
fig.show()

*The correlation observed between income and purchases of Meat Products as well as Wines suggests that income may have an influence on the number of purchases in these two product categories.*